# How to implement a YOLO (v3) object detector from scratch in PyTorch
# 1. Hiểu cách hoạt động của YOLO
Phát hiện đối tượng là một lĩnh vực đã được hưởng lợi rất nhiều từ những phát triển gần đây trong học sâu. Những năm gần đây người ta đã phát triển nhiều thuật toán để phát hiện đối tượng, một số thuật toán trong số đó bao gồm YOLO, SSD, Mask RCNN và RetinaNet.

Cách tốt nhất để học cách phát hiện đối tượng là tự mình triển khai các thuật toán ngay từ đầu. Đây chính xác là những gì chúng tôi sẽ làm trong hướng dẫn này.

Chúng ta sẽ sử dụng PyTorch để triển khai công cụ phát hiện đối tượng dựa trên YOLO v3, một trong những thuật toán phát hiện đối tượng nhanh nhất hiện có.

Mã cho hướng dẫn này được thiết kế để chạy trên Python 3.5 và PyTorch 0.4. Nó có thể được tìm thấy toàn bộ tại  [Github](https://github.com/ayooshkathuria/YOLO_v3_tutorial_from_scratch) này.

## 1.1 Điều kiện tiên quyết
* Bạn nên hiểu cách hoạt động của mạng nơ-ron tích chập bao gồm các kiến thức về **Residual Blocks**, **Upsampling** và **bỏ qua kết nối**.
* Phát hiện đối tượng, hồi quy bounding box, IoU và ngăn chặn không tối đa ([non-maximum suppression](https://www.phamduytung.com/blog/2019-12-13-nms/)) là gì.
* Cách sử dụng PyTorch cơ bản. Bạn sẽ có thể tạo các mạng nơ-ron đơn giản một cách dễ dàng.

Tôi đã cung cấp liên kết ở cuối bài đăng để đề phòng trường hợp bạn thiếu sót trong bất bài báo nào.

## 1.2 YOLO là gì?
YOLO là viết tắt của You Only Look Once. Đó là một công cụ phát hiện đối tượng sử dụng các tính năng được học bởi một mạng nơ-ron tích chập sâu để phát hiện một đối tượng. Trước khi chạm tay vào mã, chúng ta phải hiểu cách hoạt động của YOLO.

## 1.3 Mạng thần kinh tích chập đầy đủ
YOLO chỉ sử dụng các lớp tích chập, làm cho nó trở thành một mạng tích chập hoàn toàn (FCN). Nó có 75 lớp tích chập, với các kết nối bỏ qua và các lớp lấy mẫu. Không có hình thức tổng hợp nào được sử dụng và một lớp tích chập  có bước 2 được sử dụng để lấy đặt trưng. Điều này giúp ngăn ngừa mất các tính năng cấp thấp thường được cho là do gộp chung (pooling).

Là một FCN, YOLO luôn bất biến với kích thước của hình ảnh đầu vào. Tuy nhiên, trong thực tế, chúng tôi có thể muốn gắn vào kích thước đầu vào không đổi do các vấn đề khác nhau chỉ hiển thị đầu khi chúng tôi đang triển khai thuật toán.

Một vấn đề lớn trong số những vấn đề này là nếu chúng ta muốn xử lý hình ảnh của mình theo cụm (hình ảnh theo cụm có thể được xử lý song song bởi GPU, dẫn đến tăng tốc độ), chúng ta cần có tất cả hình ảnh có chiều cao và chiều rộng cố định. Điều này là cần thiết để nối nhiều hình ảnh thành một cụm lớn (ghép nhiều tensor PyTorch thành một)

Mạng làm giảm mẫu hình ảnh bởi một yếu tố được gọi là sải bước. Ví dụ: nếu sải bước của mạng là 32, thì hình ảnh đầu vào có kích thước 416 x 416 sẽ mang lại kết quả đầu ra có kích thước 13 x 13. Nói chung, sải bước của bất kỳ lớp nào trong mạng bằng hệ số mà đầu ra của lớp nhỏ hơn hình ảnh đầu vào vào mạng.

## 1.4 Diễn giải đầu ra
Thông thường, (như trường hợp của tất cả các bộ phát hiện đối tượng), các tính năng được học bởi các lớp phức hợp được chuyển vào một bộ phân loại / hồi quy để thực hiện dự đoán phát hiện (tọa độ của các hộp giới hạn, nhãn lớp, v.v.).

Trong YOLO, dự đoán được thực hiện bằng cách sử dụng lớp tích chập 1 x 1.

Bây giờ, điều đầu tiên cần chú ý là đầu ra của chúng ta là một bản đồ đặc trưng. Vì chúng ta đã sử dụng chập 1 x 1 nên kích thước của bản đồ dự đoán chính xác bằng kích thước của bản đồ đối tượng trước đó. Trong YOLO v3, cách bạn diễn giải bản đồ dự đoán này là mỗi ô có thể dự đoán một số ô giới hạn cố định.

Nhìn sâu hơn, chúng tôi có $(B x (5 + C))$ các mục trong bản đồ đối tượng. $B$ đại diện cho số ô giới hạn mà mỗi ô có thể dự đoán. Theo bài báo, mỗi hộp giới hạn B này có thể chuyên phát hiện một loại đối tượng nhất định. Mỗi hộp giới hạn có $5 + C$ thuộc tính, mô tả tọa độ trung tâm, kích thước, điểm đối tượng và tâm sự lớp $C$ cho mỗi hộp giới hạn. YOLO v3 dự đoán 3 ô giới hạn cho mỗi ô.

Bạn mong đợi mỗi ô của bản đồ đối tượng dự đoán một đối tượng thông qua một trong các hộp giới hạn của nó nếu tâm của đối tượng nằm trong trường tiếp nhận của ô đó. (Trường tiếp nhận là vùng của hình ảnh đầu vào có thể nhìn thấy đối với ô. Hãy tham khảo liên kết trên mạng nơ-ron phức hợp để làm rõ thêm).

Điều này liên quan đến cách YOLO được đào tạo, nơi chỉ có một hộp giới hạn chịu trách nhiệm phát hiện bất kỳ đối tượng nhất định nào. Đầu tiên, chúng ta phải xác định chắc chắn ô bao quanh này thuộc về ô nào.

Để làm điều đó, chúng tôi chia hình ảnh đầu vào thành **một lưới** có kích thước bằng với kích thước của bản đồ đối tượng cuối cùng.

Chúng ta hãy xem xét một ví dụ bên dưới, trong đó hình ảnh đầu vào là 416 x 416 và sải bước của mạng là 32. Như đã chỉ ra trước đó, kích thước của bản đồ đối tượng sẽ là 13 x 13. Sau đó, chúng ta chia hình ảnh đầu vào thành 13 x 13 ô.

![](https://blog.paperspace.com/content/images/2018/04/yolo-5.png)

Sau đó, ô (trên hình ảnh đầu vào) chứa tâm của *ground truth box* của một đối tượng được chọn làm ô chịu trách nhiệm dự đoán đối tượng. Trong hình ảnh, đó là ô được đánh dấu màu đỏ, chứa tâm của *ground truth box* (được đánh dấu màu vàng).

Bây giờ, ô màu đỏ là ô thứ 7 trong hàng thứ 7 trên lưới. Bây giờ chúng tôi chỉ định ô thứ 7 ở hàng thứ 7 trên bản đồ đối tượng (ô tương ứng trên bản đồ đối tượng) làm ô chịu trách nhiệm phát hiện con chó.

## 1.5 Hộp neo (Anchor Boxes)
Dự đoán chiều rộng và chiều cao của hộp giới hạn có thể có ý nghĩa, nhưng trên thực tế, điều đó dẫn đến gradient không ổn định trong quá trình đào tạo. Thay vào đó, hầu hết các trình phát hiện đối tượng hiện đại đều dự đoán các chuyển đổi log-space, hoặc đơn giản là bù đắp cho các hộp giới hạn mặc định được xác định trước được gọi là neo (**anchors**).

Sau đó, các biến đổi này được áp dụng cho các hộp neo để có được dự đoán. YOLO v3 có ba neo, dẫn đến dự đoán ba hộp giới hạn trên mỗi ô.

Trở lại câu hỏi trước đó của chúng ta, hộp giới hạn chịu trách nhiệm phát hiện con chó sẽ là hộp có mỏ neo có IoU cao nhất với *ground truth box*.

## 1.6 Đưa ra dự đoán
Công thức sau đây mô tả cách đầu ra mạng được chuyển đổi để có được các dự đoán hộp giới hạn.

![](https://blog.paperspace.com/content/images/2018/04/Screen-Shot-2018-04-10-at-3.18.08-PM.png)

$b_x$, $b_y$, $b_w$, $b_h$ là tọa độ tâm $x$, $y$, chiều rộng và chiều cao theo dự đoán của chúng tôi. $t_x$, $t_y$, $t_w$, $t_h$ là những gì mạng đầu ra. $c_x$ và $c_y$ là tọa độ trên cùng bên trái của lưới. $p_w$ và $p_h$ là kích thước neo cho hộp.

### Tọa độ trung tâm

Lưu ý rằng chúng tôi đang chạy dự đoán tọa độ trung tâm của mình thông qua một hàm sigmoid. Điều này buộc giá trị của đầu ra phải nằm trong khoảng từ 0 đến 1.

Thông thường, YOLO không dự đoán tọa độ tuyệt đối của tâm hộp giới hạn. Nó dự đoán hiệu số là:
* Tương đối với góc trên cùng bên trái của ô lưới dự đoán đối tượng.
* Được chuẩn hóa bởi các kích thước của ô từ bản đồ đối tượng, tức là, 1.

Ví dụ, hãy xem xét trường hợp của hình ảnh con chó của chúng ta. Nếu dự đoán cho trung tâm là (0.4, 0.7), thì điều này có nghĩa là trung tâm nằm ở (6.4, 6.7) trên bản đồ đối tượng 13 x 13. (Vì tọa độ trên cùng bên trái của ô màu đỏ là (6,6)).

Nhưng điều gì sẽ xảy ra nếu tọa độ x, y được dự đoán lớn hơn một, chẳng hạn (1.2, 0.7). Điều này có nghĩa là trung tâm nằm ở (7.2, 6.7). Lưu ý rằng tâm bây giờ nằm trong ô ngay bên phải ô màu đỏ của chúng ta, hoặc ô thứ 8 trong hàng thứ 7. Điều này phá vỡ lý thuyết đằng sau YOLO bởi vì nếu chúng ta giả định rằng ô màu đỏ chịu trách nhiệm dự đoán con chó, thì trung tâm của con chó phải nằm trong ô màu đỏ, chứ không phải ở ô bên cạnh nó.

Do đó, để khắc phục sự cố này, đầu ra được chuyển qua một hàm sigmoid, chức năng này cắt đầu ra trong phạm vi từ 0 đến 1, giữ hiệu quả trung tâm trong lưới đang dự đoán.

### Kích thước của hộp biên
Kích thước của hộp giới hạn được dự đoán bằng cách áp dụng phép biến đổi không gian log cho đầu ra và sau đó nhân với một mỏ neo.

![](https://blog.paperspace.com/content/images/2018/04/yolo-regression-1.png)

Các dự đoán kết quả, $b_w$ và $b_h$, được chuẩn hóa theo chiều cao và chiều rộng của hình ảnh. (Nhãn đào tạo được chọn theo cách này). Vì vậy, nếu các dự đoán $b_x$ và $b_y$ cho hộp chứa con chó là (0.3, 0.8), thì chiều rộng và chiều cao thực tế trên bản đồ đối tượng 13 x 13 là (13 x 0.3, 13 x 0.8).

### Điểm đối tượng

Điểm đối tượng đại diện cho xác suất một đối tượng được chứa bên trong hộp giới hạn. Nó phải gần bằng 1 đối với lưới màu đỏ và các lưới lân cận, trong khi gần như 0 đối với lưới ở các góc.

Điểm đối tượng cũng được thông qua một sigmoid, vì nó được hiểu là một xác suất.

### Tín nhiệm lớp
Sự tín nhiệm của lớp đại diện cho xác suất của đối tượng được phát hiện thuộc một lớp cụ thể (Chó, mèo, chuối, ô tô, v.v.). Trước phiên bản 3, YOLO đã từng softmax điểm số của lớp.

Tuy nhiên, lựa chọn thiết kế đó đã bị loại bỏ trong phiên bản 3 và các tác giả đã chọn sử dụng sigmoid thay thế. Lý do là điểm lớp Softmaxing giả định rằng các lớp loại trừ lẫn nhau. Nói một cách đơn giản, nếu một đối tượng thuộc về một lớp, thì nó được đảm bảo rằng nó không thể thuộc về một lớp khác. Điều này đúng với cơ sở dữ liệu COCO mà chúng tôi sẽ làm cơ sở cho thuật toán phát hiện.

Tuy nhiên, giả định này có thể không đúng khi chúng ta có các lớp như Phụ nữ và con người. Đây là lý do mà các tác giả đã tránh sử dụng kích hoạt Softmax.

## 1.7 Dự đoán trên các tỉ lệ khác nhau.
YOLO v3 đưa ra dự đoán trên 3 tỉ lệ khác nhau. Lớp phát hiện được sử dụng để phát hiện tại các bản đồ đối tượng ở ba kích thước khác nhau, có các **sải bước là 32, 16, 8**. Điều này có nghĩa là, với đầu vào là 416 x 416, chúng tôi thực hiện phát hiện trên các tỷ lệ 13 x 13, 26 x 26 và 52 x 52.

Mạng lấy mẫu xuống hình ảnh đầu vào cho đến lớp phát hiện đầu tiên, nơi phát hiện được thực hiện bằng cách sử dụng bản đồ đặc trưng của lớp có sải bước 32. Hơn nữa, các lớp được lấy mẫu thêm theo hệ số 2 và được nối với bản đồ đặc trưng của các lớp trước đó có bản đồ đặc trưng giống hệt nhau các kích cỡ. Một phát hiện khác bây giờ được thực hiện ở lớp có sải bước 16. Quy trình lấy mẫu ngược tương tự được lặp lại và phát hiện cuối cùng được thực hiện ở lớp có sải bước 8.

Ở mỗi tỷ lệ, mỗi ô dự đoán 3 hộp giới hạn sử dụng 3 neo, làm cho tổng số neo được sử dụng là 9 (Các neo khác nhau đối với các tỷ lệ khác nhau)

![](https://blog.paperspace.com/content/images/2018/04/yolo_Scales-1.png)

Các tác giả báo cáo rằng điều này giúp YOLO v3 phát hiện các vật thể nhỏ tốt hơn, một phàn nàn thường xuyên với các phiên bản YOLO trước đó. Lấy mẫu ngược có thể giúp mạng tìm hiểu các tính năng chi tiết là công cụ để phát hiện các vật thể nhỏ.

## 1.8 Xử lý đầu ra
Đối với hình ảnh có kích thước $416 x 416$, YOLO dự đoán $((52 x 52) + (26 x 26) + 13 x 13)) x 3 = 10647$ hộp giới hạn. Tuy nhiên, trong trường hợp hình ảnh của chúng ta, chỉ có một đối tượng, một con chó. Làm thế nào để chúng tôi giảm các phát hiện từ 10647 xuống 1?

### Lập ngưỡng theo độ tin cậy của đối tượng
Đầu tiên, chúng tôi lọc các hộp dựa trên điểm đối tượng của chúng. Nói chung, các hộp có điểm dưới ngưỡng sẽ bị bỏ qua.

### Loại bỏ không tối đa (Non-maximum Suppression-NMS)
NMS có ý định giải quyết vấn đề phát hiện nhiều hình ảnh giống nhau. Ví dụ: tất cả 3 ô giới hạn của ô lưới màu đỏ có thể phát hiện một ô hoặc các ô liền kề có thể phát hiện cùng một đối tượng.

![](https://blog.paperspace.com/content/images/2018/04/NMS-1.png)

## 1.9 Thực hiện của chúng tôi
YOLO chỉ có thể phát hiện các đối tượng thuộc các lớp có trong tập dữ liệu được sử dụng để huấn luyện mạng. Chúng tôi sẽ sử dụng tệp trọng lượng chính thức cho máy dò của chúng tôi. Các trọng số này có được bằng cách huấn luyện mạng trên tập dữ liệu COCO, và do đó chúng tôi có thể phát hiện 80 danh mục đối tượng.

Đó là nó cho phần đầu tiên. Bài đăng này giải thích đủ về thuật toán YOLO để cho phép bạn triển khai máy dò. Tuy nhiên, nếu bạn muốn tìm hiểu sâu về cách thức hoạt động của YOLO, cách nó được đào tạo và cách hoạt động của nó so với các máy dò khác, bạn có thể đọc các tài liệu gốc, các liên kết mà tôi đã cung cấp bên dưới.

## 1.10 Đọc thêm
1. [YOLO V1: You Only Look Once: Unified, Real-Time Object Detection](https://arxiv.org/pdf/1506.02640.pdf)
2. [YOLO V2: YOLO9000: Better, Faster, Stronger](https://arxiv.org/pdf/1612.08242.pdf)
3. [YOLO V3: An Incremental Improvement](https://pjreddie.com/media/files/papers/YOLOv3.pdf)
4. [Convolutional Neural Networks](https://cs231n.github.io/convolutional-networks/)
5. [Bounding Box Regression (Appendix C)](https://arxiv.org/pdf/1311.2524.pdf)
6. [Non maximum suppresion](https://towardsdatascience.com/non-maximum-suppression-nms-93ce178e177c)

# 2 Tạo các lớp của kiến trúc mạng
## Điều kiện tiên quyết
Kiến thức làm việc cơ bản của PyTorch, bao gồm cách tạo kiến trúc tùy chỉnh với các lớp `nn.Module`, `nn.Sequential` và `torch.nn.parameter`.

## 2.1 Bắt đầu
Đầu tiên, hãy tạo một thư mục darknet chứa code cho chương trình.

Sau đó, tạo một tệp `darknet.py`. Darknet là tên của kiến trúc cơ bản của YOLO. Tệp này sẽ chứa mã tạo mạng YOLO. Chúng tôi sẽ bổ sung nó bằng một tệp có tên là `util.py` sẽ chứa mã cho các chức năng trợ giúp khác nhau. Lưu cả hai tệp này vào thư mục mã code. Bạn có thể sử dụng git để theo dõi các thay đổi.

## 2.2 Tập tin cấu hình
Mã chính thức (tác giả viết bằng ngôn ngữ C) sử dụng tệp cấu hình để xây dựng mạng. Tệp `cfg` mô tả cách bố trí của mạng, từng khối. Nếu bạn đến từ caffe framework, nó tương đương với tệp `.protxt` được sử dụng để mô tả mạng.

Chúng tôi sẽ sử dụng tệp `cfg` chính thức, do tác giả phát hành để xây dựng mạng của chúng tôi. Tải xuống từ đây và đặt nó vào một thư mục có tên cfg bên trong thư mục darknet.

Nếu bạn mở tệp cấu hình, bạn sẽ thấy một cái gì đó như thế này.
```
[convolutional]
batch_normalize=1
filters=64
size=3
stride=2
pad=1
activation=leaky

[convolutional]
batch_normalize=1
filters=32
size=1
stride=1
pad=1
activation=leaky

[convolutional]
batch_normalize=1
filters=64
size=3
stride=1
pad=1
activation=leaky

[shortcut]
from=-3
activation=linear
```

Chúng ta thấy 4 khối ở trên. Trong số đó, 3 mô tả các *lớp tích chập*, tiếp theo là một *lớp lối tắt (shortcut layer)*. Lớp lối tắt là *một kết nối bỏ qua (skip connection)*, giống như lớp được sử dụng trong ResNet. Có 5 loại lớp được sử dụng trong YOLO:

### Tích chập
```
[convolutional]
batch_normalize=1
filters=64
size=3
stride=1
pad=1
activation=leaky
```
### Đường tắt
```
[shortcut]
from=-3
activation=linear
```
Lớp lối tắt là một kết nối bỏ qua, giống với kết nối được sử dụng trong ResNet. Tham số `from= -3`, có nghĩa là đầu ra của lớp lối tắt có được bằng cách thêm các bản đồ tính năng từ lớp trước và lớp thứ 3 trở về từ lớp lối tắt.

### Mẫu ngược (Upsample)
```
[upsample]
stride=2
```
Làm tăng bản đồ đối tượng trong lớp trước bằng một hệ số sải bước bằng cách sử dụng lấy mẫu ngược song tuyến.

### Lộ trình (Route)
```
[route]
layers = -4

[route]
layers = -1, 61
```
Lớp lộ trình xứng đáng được giải thích một chút. Nó có một lớp thuộc tính có thể có một hoặc hai giá trị.

Khi thuộc tính lớp chỉ có một giá trị, nó sẽ xuất ra các bản đồ đặc trưng của lớp được lập chỉ mục theo giá trị. Trong ví dụ của chúng tôi, nó là -4, vì vậy lớp sẽ xuất bản đồ đặc trưng từ lớp thứ 4 trở về trước từ lớp Route.

Khi các lớp có hai giá trị, nó trả về các bản đồ tính năng được nối của các lớp được lập chỉ mục bởi các giá trị của nó. Trong ví dụ của chúng tôi, nó là -1, 61 và lớp sẽ xuất ra các bản đồ tính năng từ lớp trước (-1) và lớp thứ 61, được nối với nhau dọc theo kích thước chiều sâu.

### YOLO
```
[yolo]
mask = 0,1,2
anchors = 10,13,  16,30,  33,23,  30,61,  62,45,  59,119,  116,90,  156,198,  373,326
classes=80
num=9
jitter=.3
ignore_thresh = .5
truth_thresh = 1
random=1
```
Lớp YOLO tương ứng với lớp Phát hiện được mô tả trong phần 1. Các neo mô tả 9 neo, nhưng chỉ các neo được lập chỉ mục bởi các thuộc tính của thẻ mặt nạ mới được sử dụng. Ở đây, giá trị của `mask=0,1,2`, có nghĩa là các neo thứ nhất, thứ hai và thứ ba được sử dụng. Điều này có ý nghĩa vì mỗi ô của lớp phát hiện dự đoán 3 ô. Tổng cộng, chúng tôi có các lớp phát hiện ở 3 tỷ lệ, chiếm tổng cộng 9 mỏ neo.

### Net
```
[net]
#Testing
batch=1
subdivisions=1
#Training
#batch=64
#subdivisions=16
width= 416
height = 416
channels=3
momentum=0.9
decay=0.0005
angle=0
saturation = 1.5
exposure = 1.5
hue=.1
```
Có một loại khối khác được gọi là `net` trong `cfg`, nhưng tôi sẽ không gọi nó là lớp vì nó chỉ mô tả thông tin về đầu vào mạng và các tham số huấn luyện. Nó không được sử dụng trong chuyển tiếp của YOLO. Tuy nhiên, nó cung cấp cho chúng tôi thông tin như kích thước đầu vào mạng, chúng tôi sử dụng để điều chỉnh các neo trong chuyển tiếp.

## 2.3 Phân tích cú pháp tệp cấu hình
Trước khi chúng tôi bắt đầu, hãy thêm các `import` cần thiết ở đầu tệp `darknet.py`.

```
from __future__ import division

import torch 
import torch.nn as nn
import torch.nn.functional as F 
from torch.autograd import Variable
import numpy as np
```

Chúng tôi định nghĩa một hàm có tên là `parse_cfg`, hàm này lấy đường dẫn của tệp cấu hình làm đầu vào.

```
def parse_cfg(cfgfile):
    """
    Takes a configuration file
    
    Returns a list of blocks. Each blocks describes a block in the neural
    network to be built. Block is represented as a dictionary in the list
    
    """
    file = open(cfgfile, 'r')
    lines = file.read().split('\n')                        # store the lines in a list
    lines = [x for x in lines if len(x) > 0]               # get read of the empty lines 
    lines = [x for x in lines if x[0] != '#']              # get rid of comments
    lines = [x.rstrip().lstrip() for x in lines]           # get rid of fringe whitespaces
    
    block = {}
    blocks = []

    for line in lines:
        if line[0] == "[":               # This marks the start of a new block
            if len(block) != 0:          # If block is not empty, implies it is storing values of previous block.
                blocks.append(block)     # add it the blocks list
                block = {}               # re-init the block
            block["type"] = line[1:-1].rstrip()     
        else:
            key,value = line.split("=") 
            block[key.rstrip()] = value.lstrip()
    blocks.append(block)

    return blocks
```

Ý tưởng ở đây là phân tích cú pháp `cfg` và lưu trữ mọi khối dưới dạng dict. Các thuộc tính của khối và giá trị của chúng được lưu trữ dưới dạng cặp **key-value** trong dict. Khi chúng tôi phân tích cú pháp qua `cfg`, chúng tôi tiếp tục nối các phần này, được biểu thị bằng `block` các biến, vào `block` danh sách. Chương trình sẽ trả về khối này.

## 2.4 Tạo các khối xây dựng
Bây giờ chúng ta sẽ sử dụng danh sách được trả về bởi `parse_cfg` ở trên để xây dựng mô-đun PyTorch cho các khối có trong tệp cấu hình.

Chúng tôi có 5 loại lớp trong danh sách (đã đề cập ở trên). PyTorch cung cấp các lớp được tạo sẵn cho các loại `convolutional` và `upsample`. Chúng ta sẽ phải viết các mô-đun của riêng mình cho phần còn lại của các lớp bằng cách mở rộng lớp `nn.Module`.

Hàm `create_modules` nhận một khối danh sách do hàm `parse_cfg` trả về.

```
def create_modules(blocks):
    net_info = blocks[0]     #Captures the information about the input and pre-processing    
    module_list = nn.ModuleList()
    prev_filters = 3
    output_filters = []
```

Trước khi chúng tôi lặp qua danh sách các khối, chúng tôi xác định một biến `net_info` để lưu trữ thông tin về mạng.

**nn.ModuleList**

Hàm của chúng ta sẽ trả về một `nn.ModuleList`. Lớp này gần giống như một danh sách bình thường chứa các đối tượng `nn.Module`. Tuy nhiên, khi chúng ta thêm `nn.ModuleList` làm thành viên của đối tượng `nn.Module` (tức là khi chúng ta thêm mô-đun vào mạng của mình), tất cả các `parameter` của đối tượng `nn.Module` (mô-đun) bên trong `nn.ModuleList` được thêm vào dưới dạng `parameter` của đối tượng `nn.Module` (tức là mạng của chúng tôi, mà chúng tôi đang thêm `nn.ModuleList` làm thành viên).

Khi chúng ta xác định một lớp chập mới, chúng ta phải xác định kích thước của hạt nhân của nó. Trong khi chiều cao và chiều rộng của hạt nhân được cung cấp bởi tệp `cfg`, độ sâu của hạt nhân chính xác là số lượng bộ lọc (hoặc độ sâu của bản đồ tính năng) có trong lớp trước đó. Điều này có nghĩa là chúng ta **cần theo dõi số lượng bộ lọc trong lớp mà convolutional layer đang được áp dụng**. Chúng tôi sử dụng biến `pres_filter` để thực hiện việc này. Chúng tôi khởi tạo giá trị này thành 3, vì hình ảnh có 3 bộ lọc tương ứng với các kênh RGB.

Route layer mang lại các bản đồ đối tượng (có thể được nối với nhau) từ các lớp trước đó. Nếu convolutional layer ngay trước route layer, thì hạt nhân sẽ được áp dụng trên các bản đồ đặc trưng của các lớp trước đó, chính xác là những cái mà route layer mang lại. Do đó, chúng ta cần theo dõi số lượng bộ lọc không chỉ trong lớp trước mà còn của từng bộ lọc trong các lớp trước đó. Khi chúng tôi lặp lại, chúng tôi thêm số lượng bộ lọc đầu ra của mỗi khối vào danh sách `output_filters`.

Bây giờ, ý tưởng là lặp lại danh sách các khối và tạo một mô-đun PyTorch cho mỗi khối khi chúng ta tiếp tục.

```
for index, x in enumerate(blocks[1:]):
        module = nn.Sequential()

        #check the type of block
        #create a new module for the block
        #append to module_list
```

`nn.Sequential` class được sử dụng để thực thi tuần tự một số đối tượng `nn.Module`. Nếu bạn nhìn vào `cfg`, bạn sẽ nhận ra một khối có thể chứa nhiều hơn một lớp. Ví dụ, một khối kiểu `convolutional` có batch norm layer cũng như lớp kích hoạt leaky ReLU ngoài một lớp convolutional. Chúng tôi xâu chuỗi các lớp này lại với nhau bằng cách sử dụng `nn.Sequential` và đó là hàm `add_module`. Ví dụ, đây là cách chúng tôi tạo các lớp convolutional và các upsample layers.

```
if (x["type"] == "convolutional"):
            #Get the info about the layer
            activation = x["activation"]
            try:
                batch_normalize = int(x["batch_normalize"])
                bias = False
            except:
                batch_normalize = 0
                bias = True

            filters= int(x["filters"])
            padding = int(x["pad"])
            kernel_size = int(x["size"])
            stride = int(x["stride"])

            if padding:
                pad = (kernel_size - 1) // 2
            else:
                pad = 0

            #Add the convolutional layer
            conv = nn.Conv2d(prev_filters, filters, kernel_size, stride, pad, bias = bias)
            module.add_module("conv_{0}".format(index), conv)

            #Add the Batch Norm Layer
            if batch_normalize:
                bn = nn.BatchNorm2d(filters)
                module.add_module("batch_norm_{0}".format(index), bn)

            #Check the activation. 
            #It is either Linear or a Leaky ReLU for YOLO
            if activation == "leaky":
                activn = nn.LeakyReLU(0.1, inplace = True)
                module.add_module("leaky_{0}".format(index), activn)

        #If it's an upsampling layer
        #We use Bilinear2dUpsampling
        elif (x["type"] == "upsample"):
            stride = int(x["stride"])
            upsample = nn.Upsample(scale_factor = 2, mode = "bilinear")
            module.add_module("upsample_{}".format(index), upsample)
```

**Route Layer / Shortcut Layers**

Tiếp theo, chúng tôi viết mã để tạo Route Layer và Shortcut Layers.

```
        #If it is a route layer
        elif (x["type"] == "route"):
            x["layers"] = x["layers"].split(',')
            #Start  of a route
            start = int(x["layers"][0])
            #end, if there exists one.
            try:
                end = int(x["layers"][1])
            except:
                end = 0
            #Positive anotation
            if start > 0: 
                start = start - index
            if end > 0:
                end = end - index
            route = EmptyLayer()
            module.add_module("route_{0}".format(index), route)
            if end < 0:
                filters = output_filters[index + start] + output_filters[index + end]
            else:
                filters= output_filters[index + start]

        #shortcut corresponds to skip connection
        elif x["type"] == "shortcut":
            shortcut = EmptyLayer()
            module.add_module("shortcut_{}".format(index), shortcut)
```

Code để tạo Route Layer xứng đáng được giải thích một chút. Lúc đầu, chúng tôi trích xuất giá trị của thuộc tính `layer`, ép nó thành một số nguyên và lưu trữ nó trong một danh sách.

Sau đó, chúng ta có một lớp mới được gọi là `EmptyLayer`, như tên cho thấy chỉ là một lớp trống.

```
route = EmptyLayer()
```

Nó được định nghĩa là.

```
class EmptyLayer(nn.Module):
    def __init__(self):
        super(EmptyLayer, self).__init__()
```

**Empty layer**

Bây giờ, một empty layer có vẻ kỳ lạ vì nó không làm gì cả. Route Layer, giống như bất kỳ lớp nào khác thực hiện một hoạt động (chuyển tiếp lớp / ghép nối trước đó). Trong PyTorch, khi chúng ta xác định một lớp mới, chúng ta phân lớp `nn.Module` và viết hoạt động mà lớp thực hiện trong hàm `forward` của đối tượng `nn.Module`.

Để thiết kế một lớp cho khối Route, chúng ta sẽ phải xây dựng một đối tượng `nn.Module` được khởi tạo với các giá trị của các `layer` thuộc tính vì nó là thành viên. Sau đó, chúng ta có thể viết mã để nối / chuyển tiếp các bản đồ đối tượng trong hàm `forward`. Cuối cùng, chúng tôi thực thi lớp này trong hàm `forward` thứ của mạng của chúng tôi.

Nhưng do mã nối khá ngắn và đơn giản (gọi `torch.cat` trên bản đồ tính năng), việc thiết kế một lớp như trên sẽ dẫn đến sự trừu tượng không cần thiết mà chỉ làm tăng sự phức tạp. Thay vào đó, những gì chúng ta có thể làm là đặt một lớp giả thay cho một route layer được đề xuất, và sau đó thực hiện nối trực tiếp trong hàm `forward`  của đối tượng `nn.Module` đại diện cho darknet. (Nếu dòng cuối cùng không có ý nghĩa với bạn, tôi khuyên bạn nên đọc cách lớp `nn.Module` được sử dụng trong PyTorch. Liên kết ở dưới cùng)

Lớp convolutional ngay trước route layer áp dụng hạt nhân của nó cho các bản đồ tính năng (có thể được nối) từ các lớp trước đó. Đoạn mã sau cập nhật biến `filters` để giữ số lượng bộ lọc được xuất ra bởi một route layer.

```
if end < 0:
    #If we are concatenating maps
    filters = output_filters[index + start] + output_filters[index + end]
else:
    filters= output_filters[index + start]
```

Shortcut layer cũng sử dụng một empty layer, vì nó cũng thực hiện một hoạt động rất đơn giản (bổ sung). Không cần cập nhật cập nhật biến `filters ` vì nó chỉ thêm bản đồ tính năng của lớp trước vào bản đồ của lớp ngay sau.

**YOLO Layer**

Cuối cùng, chúng tôi viết mã để tạo lớp YOLO.

```
        #Yolo is the detection layer
        elif x["type"] == "yolo":
            mask = x["mask"].split(",")
            mask = [int(x) for x in mask]

            anchors = x["anchors"].split(",")
            anchors = [int(a) for a in anchors]
            anchors = [(anchors[i], anchors[i+1]) for i in range(0, len(anchors),2)]
            anchors = [anchors[i] for i in mask]

            detection = DetectionLayer(anchors)
            module.add_module("Detection_{}".format(index), detection)
```

Chúng tôi xác định một `DetectionLayer` lớp mới chứa các neo được sử dụng để phát hiện các hộp giới hạn.

Lớp phát hiện được định nghĩa là

```
class DetectionLayer(nn.Module):
    def __init__(self, anchors):
        super(DetectionLayer, self).__init__()
        self.anchors = anchors
```

Vào cuối vòng lặp, chúng tôi thực hiện một số kế toán.

```
        module_list.append(module)
        prev_filters = filters
        output_filters.append(filters)
```

Điều đó kết luận phần thân của vòng lặp. Ở cuối hàm `create_modules`, chúng ta trả về một bộ giá trị chứa `net_info` và `module_list`.

## 2.5 Kiểm tra code
Bạn có thể kiểm tra mã của mình bằng cách nhập các dòng sau vào cuối `darknet.py` và chạy tệp.

```
blocks = parse_cfg("cfg/yolov3.cfg")
print(create_modules(blocks))
```

Bạn sẽ thấy một danh sách dài, (chính xác chứa 106 mục), các phần tử trong đó sẽ giống như

```

  (9): Sequential(
     (conv_9): Conv2d (128, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
     (batch_norm_9): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True)
     (leaky_9): LeakyReLU(0.1, inplace)
   )
   (10): Sequential(
     (conv_10): Conv2d (64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
     (batch_norm_10): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True)
     (leaky_10): LeakyReLU(0.1, inplace)
   )
   (11): Sequential(
     (shortcut_11): EmptyLayer(
     )
   )
```

## 2.6 Đọc thêm
1. [PyTorch tutorial](https://pytorch.org/tutorials/beginner/deep_learning_60min_blitz.html)
2. [nn.Module, nn.Parameter classes](https://pytorch.org/tutorials/beginner/blitz/neural_networks_tutorial.html#define-the-network)
3. [nn.ModuleList and nn.Sequential](https://discuss.pytorch.org/t/when-should-i-use-nn-modulelist-and-when-should-i-use-nn-sequential/5463)

# 3. Thực thi forward pass của mạng
## 3.1 Khai báo mạng
Như tôi đã chỉ ra trước đó, chúng tôi sử dụng class `nn.Module` để xây dựng các kiến trúc tùy chỉnh trong PyTorch. Hãy để chúng tôi khai báo một mạng cho chương trình phát hiện đối tượng. Trong tệp `darknet.py`, chúng tôi thêm class sau.

```
class Darknet(nn.Module):
    def __init__(self, cfgfile):
        super(Darknet, self).__init__()
        self.blocks = parse_cfg(cfgfile)
        self.net_info, self.module_list = create_modules(self.blocks)
```

Ở đây, chúng ta đã phân lớp con của lớp `nn.Module` và đặt tên cho lớp của chúng ta là `Darknet`. Chúng tôi khởi tạo mạng với các thành viên, `block`, `net_info` và `module_list`.

## 3.2 Thực thi forward pass của mạng
Forward pass của mạng được thực hiện bằng cách ghi đè phương thức `forward` của lớp `nn.Module`.

`forward` phục vụ hai mục đích. Đầu tiên, để tính toán kết quả đầu ra và thứ hai, để biến đổi các bản đồ tính năng phát hiện đầu ra theo cách mà nó có thể được xử lý dễ dàng hơn (chẳng hạn như biến đổi chúng để các bản đồ phát hiện trên nhiều tỷ lệ có thể được nối với nhau, điều này không thể thực hiện được vì chúng có kích thước khác nhau)

```
def forward(self, x, CUDA):
    modules = self.blocks[1:]
    outputs = {}   #We cache the outputs for the route layer
```

`forward` nhận ba đối số, `self`, đầu vào `x` và `CUDA`, nếu đúng, sẽ sử dụng GPU để tăng tốc forward pass.

Ở đây, chúng tôi lặp qua `self.blocks[1:]` thay vì `self.blocks` vì phần tử đầu tiên của `self.blocks` là một khối `net` không phải là một phần của forward pass.

Vì các lớp *route* và *shortcut* cần bản đồ đầu ra từ các lớp trước, chúng tôi lưu vào bộ nhớ cache các bản đồ tính năng đầu ra của mọi lớp trong `output` dict. Quan trọng là chỉ số của các lớp và các giá trị là feature maps.

Như trường hợp với hàm `create_modules`, bây giờ chúng ta lặp qua `module_list` chứa các mô-đun của mạng. Điều cần chú ý ở đây là các mô-đun đã được nối theo thứ tự giống như chúng có trong tệp cấu hình. Điều này có nghĩa là, chúng ta có thể chỉ cần chạy đầu vào của mình qua từng mô-đun để có được đầu ra của chúng ta.

```
write = 0     #This is explained a bit later
for i, module in enumerate(modules):        
    module_type = (module["type"])
```

### Convolutional và Upsample Layers
Nếu mô-đun là một mô-đun convolution hoặc mô-đun `upsample`, đây là cách forward sẽ hoạt động.

```
        if module_type == "convolutional" or module_type == "upsample":
            x = self.module_list[i](x)
```

### Route Layer / Shortcut Layer
Nếu bạn tìm code cho lớp route, chúng ta phải tính đến hai trường hợp (như được mô tả trong phần 2). Đối với trường hợp chúng ta phải ghép hai bản đồ đối tượng, chúng ta sử dụng hàm `torch.cat` với đối số thứ hai là 1. Điều này là do chúng ta muốn nối các bản đồ đối tượng theo độ sâu. (Trong PyTorch, đầu vào và đầu ra của lớp chập có định dạng `B X C X H X W`. Độ sâu tương ứng với kích thước kênh).

```
        elif module_type == "route":
            layers = module["layers"]
            layers = [int(a) for a in layers]

            if (layers[0]) > 0:
                layers[0] = layers[0] - i

            if len(layers) == 1:
                x = outputs[i + (layers[0])]

            else:
                if (layers[1]) > 0:
                    layers[1] = layers[1] - i

                map1 = outputs[i + layers[0]]
                map2 = outputs[i + layers[1]]

                x = torch.cat((map1, map2), 1)

        elif  module_type == "shortcut":
            from_ = int(module["from"])
            x = outputs[i-1] + outputs[i+from_]
```

### YOLO (Detection Layer)
Đầu ra của YOLO là một bản đồ đối tượng tích tụ có chứa các thuộc tính hộp giới hạn dọc theo độ sâu của bản đồ đối tượng. Các hộp giới hạn thuộc tính được dự đoán bởi một ô được xếp chồng lần lượt dọc theo nhau. Vì vậy, nếu bạn phải truy cập giới hạn thứ hai của ô tại (5,6), thì bạn sẽ phải lập chỉ mục nó bằng `map[5,6, (5 + C): 2 * (5 + C)]`. Biểu mẫu này rất bất tiện cho việc xử lý đầu ra chẳng hạn như tạo ngưỡng bởi độ tin cậy của đối tượng, thêm hiệu số lưới vào các trung tâm, áp dụng neo, v.v.

Một vấn đề khác là vì việc phát hiện xảy ra ở ba tỷ lệ, kích thước của bản đồ dự đoán sẽ khác nhau. Mặc dù kích thước của ba bản đồ đối tượng địa lý là khác nhau, các hoạt động xử lý đầu ra được thực hiện trên chúng là tương tự nhau. Sẽ rất tuyệt nếu bạn phải thực hiện các hoạt động này trên một tensor duy nhất, thay vì ba tensor riêng biệt.

Để khắc phục những vấn đề này, chúng tôi giới thiệu hàm `predict_transform`.

## 3.3 Chuyển đổi đầu ra
Hàm `predict_transform` tồn tại trong tệp `util.py` và chúng tôi sẽ nhập hàm khi chúng tôi sử dụng nó trước lớp `Darknet`.

Thêm các mục nhập vào đầu trang `util.py`.

```
from __future__ import division

import torch 
import torch.nn as nn
import torch.nn.functional as F 
from torch.autograd import Variable
import numpy as np
import cv2 
```

`predict_transform` nhận 5 tham số; `prediction` (đầu ra của chúng tôi), `inp_dim` (kích thước hình ảnh đầu vào), `anchors`, `num_classes` và cờ `CUDA` tùy chọn

```
def predict_transform(prediction, inp_dim, anchors, num_classes, CUDA = True):
```

Hàm `predict_transform` nhận một bản đồ tính năng phát hiện và biến nó thành một tensor 2-D, trong đó mỗi hàng của tensor tương ứng với các thuộc tính của một hộp giới hạn, theo thứ tự sau.

![](https://blog.paperspace.com/content/images/2018/04/bbox_-2.png)

Đây là code để thực hiện chuyển đổi trên.

```
    batch_size = prediction.size(0)
    stride =  inp_dim // prediction.size(2)
    grid_size = inp_dim // stride
    bbox_attrs = 5 + num_classes
    num_anchors = len(anchors)
    
    prediction = prediction.view(batch_size, bbox_attrs*num_anchors, grid_size*grid_size)
    prediction = prediction.transpose(1,2).contiguous()
    prediction = prediction.view(batch_size, grid_size*grid_size*num_anchors, bbox_attrs)
```

Kích thước của neo phù hợp với các thuộc tính height và width của khối net. Các thuộc tính này mô tả kích thước của hình ảnh đầu vào, lớn hơn (theo hệ số bước) so với bản đồ phát hiện. Do đó, chúng ta phải chia các mỏ neo theo bước của bản đồ đối tượng địa lý phát hiện.

```
    anchors = [(a[0]/stride, a[1]/stride) for a in anchors]
```

Bây giờ, chúng ta cần biến đổi đầu ra của mình theo các phương trình mà chúng ta đã thảo luận trong Phần 1.

Sigmoid tọa độ x, y và điểm đối tượng.

```
    #Sigmoid the  centre_X, centre_Y. and object confidencce
    prediction[:,:,0] = torch.sigmoid(prediction[:,:,0])
    prediction[:,:,1] = torch.sigmoid(prediction[:,:,1])
    prediction[:,:,4] = torch.sigmoid(prediction[:,:,4])
```

Thêm hiệu số lưới vào dự đoán tọa độ trung tâm.

```
    #Add the center offsets
    grid = np.arange(grid_size)
    a,b = np.meshgrid(grid, grid)

    x_offset = torch.FloatTensor(a).view(-1,1)
    y_offset = torch.FloatTensor(b).view(-1,1)

    if CUDA:
        x_offset = x_offset.cuda()
        y_offset = y_offset.cuda()

    x_y_offset = torch.cat((x_offset, y_offset), 1).repeat(1,num_anchors).view(-1,2).unsqueeze(0)

    prediction[:,:,:2] += x_y_offset
```

Áp dụng các neo cho các kích thước của hộp giới hạn.

```
    #log space transform height and the width
    anchors = torch.FloatTensor(anchors)

    if CUDA:
        anchors = anchors.cuda()

    anchors = anchors.repeat(grid_size*grid_size, 1).unsqueeze(0)
    prediction[:,:,2:4] = torch.exp(prediction[:,:,2:4])*anchors
```

Áp dụng kích hoạt sigmoid cho điểm số của lớp

```
    prediction[:,:,5: 5 + num_classes] = torch.sigmoid((prediction[:,:, 5 : 5 + num_classes]))
```

Điều cuối cùng chúng tôi muốn làm ở đây là thay đổi kích thước bản đồ phát hiện thành kích thước của hình ảnh đầu vào. Các thuộc tính hộp giới hạn ở đây có kích thước theo bản đồ đối tượng (giả sử, 13 x 13). Nếu hình ảnh đầu vào là 416 x 416, chúng tôi nhân các thuộc tính với 32 hoặc biến sải chân.

```
    prediction[:,:,:4] *= stride
```

Điều đó kết thúc phần thân của vòng lặp.

Trả về các dự đoán ở cuối hàm.

```    
    return prediction
```    

## 3.4 Xem lại Detection Layer
Bây giờ chúng ta đã chuyển đổi các tensor đầu ra của mình, chúng ta có thể nối các bản đồ phát hiện ở ba tỷ lệ khác nhau thành một tensor lớn. Lưu ý rằng điều này không thể thực hiện được trước khi chúng tôi chuyển đổi, vì người ta không thể ghép các bản đồ đối tượng địa lý có các kích thước không gian khác nhau. Nhưng kể từ bây giờ, tensor đầu ra của chúng tôi chỉ hoạt động như một bảng với các hộp giới hạn vì nó là các hàng, việc nối là rất khả thi.

Một trở ngại theo cách của chúng ta là chúng ta không thể khởi tạo một tensor rỗng, và sau đó nối một tensor không rỗng (có hình dạng khác) với nó. Vì vậy, chúng tôi trì hoãn việc khởi tạo bộ thu (tensor giữ các phát hiện) cho đến khi chúng tôi nhận được bản đồ phát hiện đầu tiên của mình, sau đó nối với bản đồ với nó khi chúng tôi nhận được các phát hiện tiếp theo.

Lưu ý dòng `write = 0` ngay trước vòng lặp trong hàm `forward`. Cờ `write` được sử dụng để cho biết liệu chúng ta có gặp phải lần phát hiện đầu tiên hay không. Nếu ghi là 0, có nghĩa là bộ sưu tập chưa được khởi tạo. Nếu nó là 1, điều đó có nghĩa là bộ sưu tập đã được khởi tạo và chúng tôi chỉ có thể nối các bản đồ phát hiện của mình với nó.

Bây giờ, chúng tôi đã trang bị cho mình hàm `predict_transform`, chúng tôi viết mã để xử lý các bản đồ tính năng phát hiện trong hàm `forward`.

Ở đầu tệp `darknet.py` của bạn, hãy thêm nhập sau.

```    
from util import * 
```

```        
    elif module_type == 'yolo':        

            anchors = self.module_list[i][0].anchors
            #Get the input dimensions
            inp_dim = int (self.net_info["height"])

            #Get the number of classes
            num_classes = int (module["classes"])

            #Transform 
            x = x.data
            x = predict_transform(x, inp_dim, anchors, num_classes, CUDA)
            if not write:              #if no collector has been intialised. 
                detections = x
                write = 1

            else:       
                detections = torch.cat((detections, x), 1)

        outputs[i] = x
```  

Bây giờ, chỉ cần trả lại các phát hiện.

```  
    return detections
```  

## 3.5 Kiểm tra chuyển tiếp

Đây là một hàm tạo đầu vào giả. Chúng tôi sẽ chuyển đầu vào này vào mạng của chúng tôi. Trước khi chúng tôi viết hàm này, hãy lưu [hình ảnh](https://raw.githubusercontent.com/ayooshkathuria/pytorch-yolo-v3/master/dog-cycle-car.png) này vào thư mục làm việc của bạn. Nếu bạn đang sử dụng Linux, sau đó nhập.

```  
wget https://github.com/ayooshkathuria/pytorch-yolo-v3/raw/master/dog-cycle-car.png
```  

Bây giờ, hãy xác định hàm ở đầu tệp `darknet.py` của bạn như sau:

```  
def get_test_input():
    img = cv2.imread("dog-cycle-car.png")
    img = cv2.resize(img, (416,416))          #Resize to the input dimension
    img_ =  img[:,:,::-1].transpose((2,0,1))  # BGR -> RGB | H X W C -> C X H X W 
    img_ = img_[np.newaxis,:,:,:]/255.0       #Add a channel at 0 (for batch) | Normalise
    img_ = torch.from_numpy(img_).float()     #Convert to float
    img_ = Variable(img_)                     # Convert to Variable
    return img_
```  

Sau đó, chúng tôi nhập mã sau:

```  
model = Darknet("cfg/yolov3.cfg")
inp = get_test_input()
pred = model(inp, torch.cuda.is_available())
print (pred)
```  

Bạn sẽ thấy một đầu ra như thế nào.

```  
(  0  ,.,.) = 
   16.0962   17.0541   91.5104  ...     0.4336    0.4692    0.5279
   15.1363   15.2568  166.0840  ...     0.5561    0.5414    0.5318
   14.4763   18.5405  409.4371  ...     0.5908    0.5353    0.4979
               ⋱                ...             
  411.2625  412.0660    9.0127  ...     0.5054    0.4662    0.5043
  412.1762  412.4936   16.0449  ...     0.4815    0.4979    0.4582
  412.1629  411.4338   34.9027  ...     0.4306    0.5462    0.4138
[torch.FloatTensor of size 1x10647x85]
``` 

Kích thước của tensor này là `1 x 10647 x 85`. Kích thước đầu tiên là kích thước batch đơn giản là 1 vì chúng tôi đã sử dụng một hình ảnh duy nhất. Đối với mỗi hình ảnh trong một batch, chúng tôi có một bảng 10647 x 85. Hàng của mỗi bảng này đại diện cho một hộp giới hạn. (4 thuộc tính bbox, 1 điểm đối tượng và 80 điểm lớp)

Tại thời điểm này, mạng của chúng ta có trọng số ngẫu nhiên và sẽ không tạo ra kết quả chính xác. Chúng tôi cần tải một tệp weight trong mạng của mình. Chúng tôi sẽ sử dụng tệp weight chính thức cho mục đích này.

## 3.6 Downloading the Pre-trained Weights
Tải xuống tệp weight vào thư mục detector của bạn. Lấy tệp weight [tại đây](https://pjreddie.com/media/files/yolov3.weights). Hoặc nếu bạn đang sử dụng Linux,

```  
wget https://pjreddie.com/media/files/yolov3.weights
```  

## 3.7 Hiểu tệp Weights
Tệp *Weight* chính thức là tệp nhị phân chứa các trọng số được lưu trữ theo kiểu nối tiếp.

Phải hết sức cẩn thận khi đọc *weight*. Các *weight* chỉ được lưu trữ dưới dạng *float*, không có gì để hướng dẫn chúng ta xem chúng thuộc về lớp nào. Nếu bạn làm hỏng, không có gì ngăn cản bạn, chẳng hạn như weight của lớp *batch norm* vào lớp *convolutional*. Vì bạn đang đọc *float*, không có cách nào để phân biệt weight thuộc về lớp nào. Do đó, chúng ta phải hiểu cách các weight được lưu trữ.

Đầu tiên, các trọng số chỉ thuộc về hai loại lớp, hoặc lớp *batch norm* hoặc *convolutional*.

Weight cho các lớp này được lưu trữ chính xác theo thứ tự như chúng xuất hiện trong tệp cấu hình. Vì vậy, nếu một khối `convolutional` được theo sau bởi một khối `shortcut` và sau đó là khối `shortcut` bởi một khối `convolutional` khác, Bạn sẽ mong đợi tệp chứa các weight của khối `convolutional` trước đó, tiếp theo là các weight của khối `convolutional` sau.

Khi lớp `batch norm` xuất hiện trong một khối `convolutional`, không có sai lệch. Tuy nhiên, khi không có lớp batch norm, "weight" thiên vị phải đọc từ tệp.

Sơ đồ sau đây tổng hợp cách lưu trữ các weight.

![](https://blog.paperspace.com/content/images/2018/04/wts-1.png)

## 3.8 Tải Weights
Hãy để chúng tôi viết một hàm tải weight. Nó sẽ là một chức năng thành viên của lớp `Darknet`. Nó sẽ lấy một đối số khác với `self`, đường dẫn của `weightfile`.

```
def load_weights(self, weightfile):
```

160 byte đầu tiên của tệp weights lưu trữ 5 giá trị int32 tạo thành tiêu đề của tệp.

```
    #Open the weights file
    fp = open(weightfile, "rb")

    #The first 5 values are header information 
    # 1. Major version number
    # 2. Minor Version Number
    # 3. Subversion number 
    # 4,5. Images seen by the network (during training)
    header = np.fromfile(fp, dtype = np.int32, count = 5)
    self.header = torch.from_numpy(header)
    self.seen = self.header[3]
```

Phần còn lại của các bit bây giờ đại diện cho trọng số, theo thứ tự được mô tả ở trên. Các trọng số được lưu trữ dưới dạng `float32` hoặc float 32-bit. Hãy tải phần còn lại của các trọng số trong một `np.ndarray`.

```
    ptr = 0
    for i in range(len(self.module_list)):
        module_type = self.blocks[i + 1]["type"]

        #If module_type is convolutional load weights
        #Otherwise ignore
```

Trong vòng lặp, trước tiên chúng ta kiểm tra xem khối `convolutional` có `batch_normalise` True hay không. Dựa vào đó, chúng tôi tải weight.

```        
        if module_type == "convolutional":
            model = self.module_list[i]
            try:
                batch_normalize = int(self.blocks[i+1]["batch_normalize"])
            except:
                batch_normalize = 0

            conv = model[0]
```

Chúng tôi giữ một biến có tên `ptr` để theo dõi vị trí của chúng tôi trong mảng trọng số. Bây giờ, nếu `batch_normalize` là True, chúng ta tải trọng số như sau.

```   
    if (batch_normalize):
            bn = model[1]

            #Get the number of weights of Batch Norm Layer
            num_bn_biases = bn.bias.numel()

            #Load the weights
            bn_biases = torch.from_numpy(weights[ptr:ptr + num_bn_biases])
            ptr += num_bn_biases

            bn_weights = torch.from_numpy(weights[ptr: ptr + num_bn_biases])
            ptr  += num_bn_biases

            bn_running_mean = torch.from_numpy(weights[ptr: ptr + num_bn_biases])
            ptr  += num_bn_biases

            bn_running_var = torch.from_numpy(weights[ptr: ptr + num_bn_biases])
            ptr  += num_bn_biases

            #Cast the loaded weights into dims of model weights. 
            bn_biases = bn_biases.view_as(bn.bias.data)
            bn_weights = bn_weights.view_as(bn.weight.data)
            bn_running_mean = bn_running_mean.view_as(bn.running_mean)
            bn_running_var = bn_running_var.view_as(bn.running_var)

            #Copy the data to model
            bn.bias.data.copy_(bn_biases)
            bn.weight.data.copy_(bn_weights)
            bn.running_mean.copy_(bn_running_mean)
            bn.running_var.copy_(bn_running_var)
```

Nếu batch_norm không đúng, chỉ cần tải các biases của lớp convolution.

```
        else:
            #Number of biases
            num_biases = conv.bias.numel()

            #Load the weights
            conv_biases = torch.from_numpy(weights[ptr: ptr + num_biases])
            ptr = ptr + num_biases

            #reshape the loaded weights according to the dims of the model weights
            conv_biases = conv_biases.view_as(conv.bias.data)

            #Finally copy the data
            conv.bias.data.copy_(conv_biases)
```

Cuối cùng, chúng tôi tải trọng số của lớp convolutional cuối cùng.

```
#Let us load the weights for the Convolutional layers
num_weights = conv.weight.numel()

#Do the same as above for weights
conv_weights = torch.from_numpy(weights[ptr:ptr+num_weights])
ptr = ptr + num_weights

conv_weights = conv_weights.view_as(conv.weight.data)
conv.weight.data.copy_(conv_weights)
```

Chúng ta đã thực hiện xong hàm này và bây giờ bạn có thể tải trọng số trong đối tượng Darknet của mình bằng cách gọi hàm `load_weights` trên đối tượng `darknet`.

```
model = Darknet("cfg/yolov3.cfg")
model.load_weights("yolov3.weights")
```

# 4. Ngưỡng tin cậy và Non-maximum Suppression (Non-maximum Suppression)
Trong các phần trước, chúng tôi đã xây dựng một mô hình phát hiện đối tượng với một hình ảnh đầu vào. Nói một cách chính xác, đầu ra của chúng ta là một tensor có kích thước $B x 10647 x 85$. $B$ là số hình ảnh trong một batch, $10647$ là số hộp giới hạn được dự đoán trên mỗi hình ảnh và $85$ là số thuộc tính hộp giới hạn.

Tuy nhiên, như mô tả trong phần 1, chúng ta phải đặt đầu ra của mình theo ngưỡng điểm đối tượng và *triệt tiêu không tối đa* (**Non-maximum Suppression**), để có được những gì tôi sẽ gọi trong phần còn lại của bài đăng này là phát hiện đúng. Để làm điều đó, chúng tôi sẽ tạo một hàm có tên là `write_results` trong tệp tin `use.py`.

```
def write_results(prediction, confidence, num_classes, nms_conf = 0.4):
```

Các hàm nhận đầu vào là dự đoán, độ tin cậy (ngưỡng điểm của đối tượng), `num_classes` (trong trường hợp của chúng tôi là 80) và `nms_conf` (ngưỡng NMS IoU).

## 4.1 Ngưỡng tin cậy đối tượng

Dự đoán của chúng tôi chứa thông tin về các hộp giới hạn $B x 10647$. Đối với mỗi hộp giới hạn có điểm đối tượng dưới ngưỡng, chúng tôi đặt các giá trị của mọi thuộc tính của nó (toàn bộ hàng đại diện cho hộp giới hạn) thành 0.

```
conf_mask = (prediction[:,:,4] > confidence).float().unsqueeze(2)
prediction = prediction*conf_mask
```

## 4.2 Thực hiện ngăn chặn không tối đa

   >Lưu ý: Tôi giả sử bạn hiểu **IoU** (Giao nhau qua liên hiệp) là gì và **Non-maximun Suppression**. Nếu không phải như vậy, hãy tham khảo các liên kết ở cuối bài viết).

Các thuộc tính hộp giới hạn mà chúng ta có bây giờ được mô tả bằng tọa độ trung tâm, cũng như chiều cao và chiều rộng của hộp giới hạn. Tuy nhiên, sẽ dễ dàng hơn để tính IoU của hai hộp, sử dụng tọa độ của một cặp góc chéo của mỗi hộp. Vì vậy, chúng tôi biến đổi các thuộc tính (**center x, center y, height, width**) của các hộp của chúng tôi thành (**góc trên bên trái x, góc trên bên trái y, góc dưới cùng bên phải x, góc dưới cùng bên phải y**).

```
    box_corner = prediction.new(prediction.shape)
    box_corner[:,:,0] = (prediction[:,:,0] - prediction[:,:,2]/2)
    box_corner[:,:,1] = (prediction[:,:,1] - prediction[:,:,3]/2)
    box_corner[:,:,2] = (prediction[:,:,0] + prediction[:,:,2]/2) 
    box_corner[:,:,3] = (prediction[:,:,1] + prediction[:,:,3]/2)
    prediction[:,:,:4] = box_corner[:,:,:4]
```

Số lượng phát hiện thực trong mỗi hình ảnh có thể khác nhau. Ví dụ: batch thước 3 trong đó hình ảnh 1, 2 và 3 có 5, 2, 4 phát hiện đúng tương ứng. Do đó, ngưỡng tin cậy và NMS phải được thực hiện cho một hình ảnh cùng một lúc. Điều này có nghĩa là, chúng ta không thể vecto hóa các hoạt động liên quan và phải lặp lại chiều đầu tiên của `prediction` (chứa các chỉ mục của hình ảnh trong một lô).

```
    batch_size = prediction.size(0)

    write = False

    for ind in range(batch_size):
        image_pred = prediction[ind]          #image Tensor
           #confidence threshholding 
           #NMS
```   

Như đã mô tả trước đây, cờ `write` được sử dụng để chỉ ra rằng chúng tôi chưa khởi tạo đầu ra, một tensor mà chúng tôi sẽ sử dụng để thu thập các phát hiện đúng trên toàn bộ batch.

Khi bên trong vòng lặp, hãy dọn dẹp mọi thứ một chút. Lưu ý rằng mỗi hàng hộp giới hạn có 85 thuộc tính, trong đó 80 thuộc tính là scores của lớp. Tại thời điểm này, chúng tôi chỉ quan tâm đến điểm lớp có giá trị lớn nhất. Vì vậy, chúng tôi xóa 80 điểm lớp khỏi mỗi hàng và thay vào đó, thêm chỉ số của lớp có giá trị lớn nhất, cũng như điểm số của lớp đó.

```
        max_conf, max_conf_score = torch.max(image_pred[:,5:5+ num_classes], 1)
        max_conf = max_conf.float().unsqueeze(1)
        max_conf_score = max_conf_score.float().unsqueeze(1)
        seq = (image_pred[:,:5], max_conf, max_conf_score)
        image_pred = torch.cat(seq, 1)
```

Hãy nhớ rằng chúng tôi đã đặt các hàng hộp giới hạn có độ tin cậy đối tượng nhỏ hơn ngưỡng thành 0? Hãy loại bỏ chúng.

```
        non_zero_ind =  (torch.nonzero(image_pred[:,4]))
        try:
            image_pred_ = image_pred[non_zero_ind.squeeze(),:].view(-1,7)
        except:
            continue
        
        #For PyTorch 0.4 compatibility
        #Since the above code with not raise exception for no detection 
        #as scalars are supported in PyTorch 0.4
        if image_pred_.shape[0] == 0:
            continue
```

Khối thử loại trừ ở đó để xử lý các tình huống mà chúng tôi không có phát hiện. Trong trường hợp đó, chúng tôi sử dụng tiếp tục để bỏ qua phần còn lại của phần thân vòng lặp cho hình ảnh này.

Bây giờ, chúng ta hãy phát hiện các lớp trong một hình ảnh.

```
        #Get the various classes detected in the image
        img_classes = unique(image_pred_[:,-1]) # -1 index holds the class index
```

Vì có thể có nhiều phát hiện đúng của cùng một lớp, chúng tôi sử dụng một hàm được gọi là `unique` để nhận các lớp có mặt trong bất kỳ hình ảnh nhất định nào.

```
def unique(tensor):
    tensor_np = tensor.cpu().numpy()
    unique_np = np.unique(tensor_np)
    unique_tensor = torch.from_numpy(unique_np)
    
    tensor_res = tensor.new(unique_tensor.shape)
    tensor_res.copy_(unique_tensor)
    return tensor_res

```

Sau đó, chúng tôi thực hiện NMS theo phân lớp.

```
        for cls in img_classes:
            #perform NMS
```

Khi chúng ta đã ở bên trong vòng lặp, điều đầu tiên chúng ta làm là trích xuất các phát hiện của một lớp cụ thể (được biểu thị bằng biến `cls`).

```
        #get the detections with one particular class
        cls_mask = image_pred_*(image_pred_[:,-1] == cls).float().unsqueeze(1)
        class_mask_ind = torch.nonzero(cls_mask[:,-2]).squeeze()
        image_pred_class = image_pred_[class_mask_ind].view(-1,7)

        #sort the detections such that the entry with the maximum objectness
        #confidence is at the top
        conf_sort_index = torch.sort(image_pred_class[:,4], descending = True )[1]
        image_pred_class = image_pred_class[conf_sort_index]
        idx = image_pred_class.size(0)   #Number of detections
```

Bây giờ, chúng tôi thực hiện NMS.

```
    for i in range(idx):
        #Get the IOUs of all boxes that come after the one we are looking at 
        #in the loop
        try:
            ious = bbox_iou(image_pred_class[i].unsqueeze(0), image_pred_class[i+1:])
        except ValueError:
            break

        except IndexError:
            break

        #Zero out all the detections that have IoU > treshhold
        iou_mask = (ious < nms_conf).float().unsqueeze(1)
        image_pred_class[i+1:] *= iou_mask       

        #Remove the non-zero entries
        non_zero_ind = torch.nonzero(image_pred_class[:,4]).squeeze()
        image_pred_class = image_pred_class[non_zero_ind].view(-1,7)

```

Ở đây, chúng tôi sử dụng một hàm `bbox_iou`. Đầu vào đầu tiên là hàng hộp giới hạn được lập chỉ mục bởi biến `i` trong vòng lặp.

Đầu vào thứ hai cho `bbox_iou` là một hàng chục của nhiều hàng hộp giới hạn. Đầu ra của hàm `bbox_iou` là một tensor chứa IoU của hộp giới hạn được đại diện bởi đầu vào đầu tiên với mỗi hộp giới hạn có trong đầu vào thứ hai.

![](https://blog.paperspace.com/content/images/2018/04/bbox-3.png)

Nếu chúng ta có hai hộp giới hạn của cùng một lớp có IoU lớn hơn một ngưỡng, thì hộp có độ tin cậy của lớp thấp hơn sẽ bị loại. Chúng tôi đã sắp xếp các hộp giới hạn với những hộp có tâm sự cao hơn ở trên cùng.

Trong phần nội dung của vòng lặp, các dòng sau cung cấp IoU của hộp, được lập chỉ mục bởi `i` với tất cả các hộp giới hạn có chỉ số cao hơn `i`.

```
ious = bbox_iou(image_pred_class[i].unsqueeze(0), image_pred_class[i+1:])
```

Mỗi lần lặp, nếu bất kỳ hộp giới hạn nào có chỉ số lớn hơn tôi có IoU (với hộp được lập chỉ mục bởi `i`) lớn hơn ngưỡng `nms_thresh`, thì hộp cụ thể đó sẽ bị loại bỏ.

```
#Zero out all the detections that have IoU > treshhold
iou_mask = (ious < nms_conf).float().unsqueeze(1)
image_pred_class[i+1:] *= iou_mask       

#Remove the non-zero entries
non_zero_ind = torch.nonzero(image_pred_class[:,4]).squeeze()
image_pred_class = image_pred_class[non_zero_ind]         
```

Cũng lưu ý rằng, chúng tôi đã đặt dòng mã để tính toán `ious` trong một khối `try-catch`. Điều này là do vòng lặp được thiết kế để chạy các lần lặp `idx` (số hàng trong `image_pred_class`). Tuy nhiên, khi chúng tôi tiếp tục với vòng lặp, một số hộp giới hạn có thể bị xóa khỏi `image_pred_class`. Điều này có nghĩa là, ngay cả khi một giá trị bị xóa khỏi `image_pred_class`, chúng ta không thể có các lần lặp `idx`. Do đó, chúng tôi có thể cố gắng lập chỉ mục một giá trị nằm ngoài giới hạn (`IndexError`) hoặc slice `image_pred_class [i + 1:]` có thể trả về một tensor trống, gán giá trị này sẽ kích hoạt một `ValueError`. Tại thời điểm đó, chúng tôi có thể chắc chắn rằng NMS có thể loại bỏ không có hộp giới hạn nào nữa và chúng tôi thoát khỏi vòng lặp.

## 4.3 Tính IoU

Đây là hàm `bbox_iou`.

```

def bbox_iou(box1, box2):
    """
    Returns the IoU of two bounding boxes 
    
    
    """
    #Get the coordinates of bounding boxes
    b1_x1, b1_y1, b1_x2, b1_y2 = box1[:,0], box1[:,1], box1[:,2], box1[:,3]
    b2_x1, b2_y1, b2_x2, b2_y2 = box2[:,0], box2[:,1], box2[:,2], box2[:,3]
    
    #get the corrdinates of the intersection rectangle
    inter_rect_x1 =  torch.max(b1_x1, b2_x1)
    inter_rect_y1 =  torch.max(b1_y1, b2_y1)
    inter_rect_x2 =  torch.min(b1_x2, b2_x2)
    inter_rect_y2 =  torch.min(b1_y2, b2_y2)
    
    #Intersection area
    inter_area = torch.clamp(inter_rect_x2 - inter_rect_x1 + 1, min=0) * torch.clamp(inter_rect_y2 - inter_rect_y1 + 1, min=0)
 
    #Union Area
    b1_area = (b1_x2 - b1_x1 + 1)*(b1_y2 - b1_y1 + 1)
    b2_area = (b2_x2 - b2_x1 + 1)*(b2_y2 - b2_y1 + 1)
    
    iou = inter_area / (b1_area + b2_area - inter_area)
    
    return iou

```

## 4.4 Viết chương trình dự đoán

Hàm `write_results` xuất ra một tensor có kích thước D x 8. Ở đây D là các phát hiện đúng trong tất cả các hình ảnh, mỗi hình được biểu diễn bằng một hàng. Mỗi phát hiện có 8 thuộc tính, cụ thể là chỉ số của hình ảnh trong lô mà phát hiện thuộc về, **tọa độ 4 góc, điểm đối tượng, điểm của lớp có độ tin cậy tối đa và chỉ số của lớp đó**.

Cũng giống như trước đây, chúng tôi không khởi tạo tensor đầu ra trừ khi chúng tôi có một phát hiện để gán cho nó. Khi nó đã được khởi tạo, chúng tôi kết hợp các phát hiện tiếp theo với nó. Chúng tôi sử dụng cờ `write` để cho biết tensor đã được khởi tạo hay chưa. Vào cuối vòng lặp lặp qua các lớp, chúng tôi thêm các phát hiện kết quả vào tensor `output`.

```
       batch_ind = image_pred_class.new(image_pred_class.size(0), 1).fill_(ind)      
       #Repeat the batch_id for as many detections of the class cls in the image
       seq = batch_ind, image_pred_class

       if not write:
           output = torch.cat(seq,1)
           write = True
       else:
           out = torch.cat(seq,1)
           output = torch.cat((output,out))
```

Khi kết thúc hàm, chúng tôi kiểm tra xem đầu ra đã được khởi tạo hay chưa. Nếu không có nghĩa là không có một phát hiện nào trong bất kỳ hình ảnh nào của lô. Trong trường hợp đó, chúng tôi trả về 0.

```
    try:
        return output
    except:
        return 0
```

# 5. Thiết kế đường ống đầu vào và đầu ra
Trong phần này, chúng tôi sẽ xây dựng các đường ống đầu vào và đầu ra của chương trình của chúng tôi. Điều này liên quan đến việc đọc hình ảnh trên đĩa, đưa ra dự đoán, sử dụng dự đoán để vẽ các hộp giới hạn trên hình ảnh, sau đó lưu chúng vào đĩa. Chúng tôi cũng sẽ đề cập đến cách để thiết bị phát hiện hoạt động trong thời gian thực trên nguồn cấp dữ liệu máy ảnh hoặc video. Chúng tôi sẽ giới thiệu một số cờ dòng lệnh để cho phép một số thử nghiệm với các siêu tham số khác nhau của mạng. Vì vậy, hãy bắt đầu.

>chúng ta phải cài Opencv cho phần này

Create a file `detector.py` in tour detector file. Add neccasary imports at top of it.

```
from __future__ import division
import time
import torch 
import torch.nn as nn
from torch.autograd import Variable
import numpy as np
import cv2 
from util import *
import argparse
import os 
import os.path as osp
from darknet import Darknet
import pickle as pkl
import pandas as pd
import random

```

## 5.1 Tạo đối số
Vì detector.py là tệp mà chúng tôi sẽ thực thi để chạy trình của mình, thật tuyệt khi có các đối số dòng lệnh mà chúng tôi có thể chuyển vào nó. Tôi đã sử dụng mô-đun ArgParse của python để làm điều đó.

```
def arg_parse():
    """
    Parse arguements to the detect module
    
    """
    
    parser = argparse.ArgumentParser(description='YOLO v3 Detection Module')
   
    parser.add_argument("--images", dest = 'images', help = 
                        "Image / Directory containing images to perform detection upon",
                        default = "imgs", type = str)
    parser.add_argument("--det", dest = 'det', help = 
                        "Image / Directory to store detections to",
                        default = "det", type = str)
    parser.add_argument("--bs", dest = "bs", help = "Batch size", default = 1)
    parser.add_argument("--confidence", dest = "confidence", help = "Object Confidence to filter predictions", default = 0.5)
    parser.add_argument("--nms_thresh", dest = "nms_thresh", help = "NMS Threshhold", default = 0.4)
    parser.add_argument("--cfg", dest = 'cfgfile', help = 
                        "Config file",
                        default = "cfg/yolov3.cfg", type = str)
    parser.add_argument("--weights", dest = 'weightsfile', help = 
                        "weightsfile",
                        default = "yolov3.weights", type = str)
    parser.add_argument("--reso", dest = 'reso', help = 
                        "Input resolution of the network. Increase to increase accuracy. Decrease to increase speed",
                        default = "416", type = str)
    
    return parser.parse_args()
    
args = arg_parse()
images = args.images
batch_size = int(args.bs)
confidence = float(args.confidence)
nms_thesh = float(args.nms_thresh)
start = 0
CUDA = torch.cuda.is_available()
```

Trong số này, các cờ quan trọng là `images` (được sử dụng để chỉ định hình ảnh đầu vào hoặc thư mục của hình ảnh), `det` (Thư mục để lưu các phát hiện vào), `reso` (Độ phân giải của hình ảnh đầu vào, có thể được sử dụng để cân bằng tốc độ - độ chính xác), `cfg` (tệp cấu hình thay thế ) và `weightfile`.

## 5.2 Loading the Network
Tải xuống tệp `coco.names` từ [đây](https://raw.githubusercontent.com/ayooshkathuria/YOLO_v3_tutorial_from_scratch/master/data/coco.names), tệp chứa tên của các đối tượng trong tập dữ liệu COCO. Tạo một thư mục `data` trong thư mục chương trình của bạn. Tương tự, nếu bạn đang sử dụng Linux, bạn có thể nhập.

```
mkdir data
cd data
https://raw.githubusercontent.com/ayooshkathuria/YOLO_v3_tutorial_from_scratch/master/data/coco.names
```

Sau đó, chúng tôi tải tệp lớp trong chương trình của chúng tôi.

```
num_classes = 80    #For COCO
classes = load_classes("data/coco.names")
```

`load_classes` là một hàm được định nghĩa trong `use.py` trả về một từ điển ánh xạ chỉ mục của mọi lớp thành một chuỗi tên của lớp đó.

```
def load_classes(namesfile):
    fp = open(namesfile, "r")
    names = fp.read().split("\n")[:-1]
    return names
```

Khởi tạo mạng và tải trọng số.

```#Set up the neural network
print("Loading network.....")
model = Darknet(args.cfgfile)
model.load_weights(args.weightsfile)
print("Network successfully loaded")

model.net_info["height"] = args.reso
inp_dim = int(model.net_info["height"])
assert inp_dim % 32 == 0 
assert inp_dim > 32

#If there's a GPU availible, put the model on GPU
if CUDA:
    model.cuda()

#Set the model in evaluation mode
model.eval()

```

## 5.3 Đọc hình ảnh đầu vào
Đọc hình ảnh từ đĩa hoặc hình ảnh từ một thư mục. Các đường dẫn của hình ảnh / hình ảnh được lưu trữ trong một danh sách gọi là `imlist`.

```
read_dir = time.time()
#Detection phase
try:
    imlist = [osp.join(osp.realpath('.'), images, img) for img in os.listdir(images)]
except NotADirectoryError:
    imlist = []
    imlist.append(osp.join(osp.realpath('.'), images))
except FileNotFoundError:
    print ("No file or directory with the name {}".format(images))
    exit()
```

`read_dir` là một trạm kiểm soát dùng để đo thời gian. (Chúng ta sẽ gặp một số trong số này)

Nếu thư mục để lưu các phát hiện, được xác định bởi cờ `det`, không tồn tại, hãy tạo nó.

```
if not os.path.exists(args.det):
    os.makedirs(args.det)
```

We will use OpenCV to load the images.

```
load_batch = time.time()
loaded_ims = [cv2.imread(x) for x in imlist]
```

load_batch lại là mộtcheckpoint.

OpenCV tải một hình ảnh dưới dạng một mảng màu, với BGR là thứ tự của các kênh màu. Định dạng đầu vào hình ảnh của PyTorch là (Batches x Channels x Height x Width), với thứ tự kênh là RGB. Do đó, chúng tôi viết hàm `prep_image` trong `use.py` để biến đổi mảng numpy thành định dạng đầu vào của PyTorch.

Trước khi chúng ta có thể viết hàm này, chúng ta phải viết một hàm `letterbox_image` để thay đổi kích thước hình ảnh của chúng ta, giữ cho tỷ lệ khung hình nhất quán và đệm các vùng còn lại bằng màu (128,128,128)

```
def letterbox_image(img, inp_dim):
    '''resize image with unchanged aspect ratio using padding'''
    img_w, img_h = img.shape[1], img.shape[0]
    w, h = inp_dim
    new_w = int(img_w * min(w/img_w, h/img_h))
    new_h = int(img_h * min(w/img_w, h/img_h))
    resized_image = cv2.resize(img, (new_w,new_h), interpolation = cv2.INTER_CUBIC)
    
    canvas = np.full((inp_dim[1], inp_dim[0], 3), 128)

    canvas[(h-new_h)//2:(h-new_h)//2 + new_h,(w-new_w)//2:(w-new_w)//2 + new_w,  :] = resized_image
    
    return canvas
```

Bây giờ, chúng ta viết hàm lấy hình ảnh OpenCV và chuyển nó thành đầu vào của mạng của chúng ta.

```
def prep_image(img, inp_dim):
    """
    Prepare image for inputting to the neural network. 
    
    Returns a Variable 
    """

    img = cv2.resize(img, (inp_dim, inp_dim))
    img = img[:,:,::-1].transpose((2,0,1)).copy()
    img = torch.from_numpy(img).float().div(255.0).unsqueeze(0)
    return img
```

Ngoài hình ảnh đã biến đổi, chúng tôi cũng duy trì một danh sách các hình ảnh gốc và `im_dim_list`, một danh sách chứa các kích thước của hình ảnh gốc.

```
#PyTorch Variables for images
im_batches = list(map(prep_image, loaded_ims, [inp_dim for x in range(len(imlist))]))

#List containing dimensions of original images
im_dim_list = [(x.shape[1], x.shape[0]) for x in loaded_ims]
im_dim_list = torch.FloatTensor(im_dim_list).repeat(1,2)

if CUDA:
    im_dim_list = im_dim_list.cuda()
```

## 5.4 Tạo các batches
```
leftover = 0
if (len(im_dim_list) % batch_size):
   leftover = 1

if batch_size != 1:
   num_batches = len(imlist) // batch_size + leftover            
   im_batches = [torch.cat((im_batches[i*batch_size : min((i +  1)*batch_size,
                       len(im_batches))]))  for i in range(num_batches)]
```

## 5.5 Vòng lặp phát hiện
Chúng tôi lặp lại các lô, tạo ra dự đoán và nối các tensors dự đoán (hình dạng, D x 8, đầu ra của hàm `write_results`) của tất cả các hình ảnh mà chúng tôi có để thực hiện phát hiện.

Đối với mỗi lô, chúng tôi đo thời gian thực hiện để phát hiện là thời gian dành cho giữa việc lấy đầu vào và tạo ra đầu ra của hàm `write_results`. Trong đầu ra được trả về bởi `write_prediction`, một trong những thuộc tính là chỉ mục của hình ảnh theo lô. Chúng tôi biến đổi thuộc tính cụ thể đó theo cách mà bây giờ nó đại diện cho chỉ mục của hình ảnh trong `imlist`, danh sách chứa địa chỉ của tất cả các hình ảnh.

Sau đó, chúng tôi in thời gian thực hiện cho mỗi lần phát hiện cũng như đối tượng được phát hiện trong mỗi hình ảnh.

Nếu đầu ra của hàm `write_results` cho lô là `int(0)`, nghĩa là không có phát hiện, chúng ta sử dụng `continous` để bỏ qua vòng lặp còn lại.

```
write = 0
start_det_loop = time.time()
for i, batch in enumerate(im_batches):
    #load the image 
    start = time.time()
    if CUDA:
        batch = batch.cuda()

    prediction = model(Variable(batch, volatile = True), CUDA)

    prediction = write_results(prediction, confidence, num_classes, nms_conf = nms_thesh)

    end = time.time()

    if type(prediction) == int:

        for im_num, image in enumerate(imlist[i*batch_size: min((i +  1)*batch_size, len(imlist))]):
            im_id = i*batch_size + im_num
            print("{0:20s} predicted in {1:6.3f} seconds".format(image.split("/")[-1], (end - start)/batch_size))
            print("{0:20s} {1:s}".format("Objects Detected:", ""))
            print("----------------------------------------------------------")
        continue

    prediction[:,0] += i*batch_size    #transform the atribute from index in batch to index in imlist 

    if not write:                      #If we have't initialised output
        output = prediction  
        write = 1
    else:
        output = torch.cat((output,prediction))

    for im_num, image in enumerate(imlist[i*batch_size: min((i +  1)*batch_size, len(imlist))]):
        im_id = i*batch_size + im_num
        objs = [classes[int(x[-1])] for x in output if int(x[0]) == im_id]
        print("{0:20s} predicted in {1:6.3f} seconds".format(image.split("/")[-1], (end - start)/batch_size))
        print("{0:20s} {1:s}".format("Objects Detected:", " ".join(objs)))
        print("----------------------------------------------------------")

    if CUDA:
        torch.cuda.synchronize() 
```

Dòng `torch.cuda.synchronize` đảm bảo rằng nhân CUDA được đồng bộ hóa với CPU. Nếu không, nhân CUDA trả lại quyền điều khiển cho CPU ngay sau khi công việc GPU được xếp hàng đợi và trước khi công việc GPU hoàn thành (Gọi không đồng bộ). Điều này có thể dẫn đến sai thời gian nếu `end = time.time()` được in trước khi công việc GPU thực sự kết thúc.

Bây giờ, chúng tôi có khả năng phát hiện tất cả các hình ảnh trong Đầu ra Tensor của chúng tôi. Hãy để chúng tôi vẽ các hộp giới hạn trên hình ảnh.

## 5.6 Vẽ các hộp giới hạn trên hình ảnh
Chúng tôi sử dụng một khối thử bắt để kiểm tra xem đã có một phát hiện duy nhất nào được thực hiện hay chưa. Nếu không đúng như vậy, hãy thoát khỏi chương trình.

```
try:
    output
except NameError:
    print ("No detections were made")
    exit()
```

Trước khi chúng tôi vẽ các hộp giới hạn, các dự đoán có trong bộ đầu ra của chúng tôi tuân theo kích thước đầu vào của mạng chứ không phải kích thước ban đầu của hình ảnh. Vì vậy, trước khi chúng ta có thể vẽ các hộp giới hạn, chúng ta hãy biến đổi các thuộc tính góc của mỗi hộp giới hạn thành kích thước ban đầu của hình ảnh.

Trước khi chúng tôi vẽ các hộp giới hạn, các dự đoán có trong tensor đầu ra của chúng tôi là các dự đoán trên hình ảnh đệm chứ không phải hình ảnh gốc. Chỉ điều chỉnh lại tỷ lệ chúng theo kích thước của hình ảnh đầu vào sẽ không hoạt động ở đây. Trước tiên, chúng ta cần biến đổi tọa độ của các hộp được đo đối với ranh giới của vùng trên hình ảnh đệm có chứa hình ảnh gốc.

```
im_dim_list = torch.index_select(im_dim_list, 0, output[:,0].long())

scaling_factor = torch.min(inp_dim/im_dim_list,1)[0].view(-1,1)


output[:,[1,3]] -= (inp_dim - scaling_factor*im_dim_list[:,0].view(-1,1))/2
output[:,[2,4]] -= (inp_dim - scaling_factor*im_dim_list[:,1].view(-1,1))/2
```

Bây giờ, tọa độ của chúng tôi phù hợp với kích thước của hình ảnh của chúng tôi trên khu vực được đệm. Tuy nhiên, trong hàm `letterbox_image`, chúng tôi đã thay đổi kích thước cả hai kích thước của hình ảnh bằng hệ số tỷ lệ (hãy nhớ cả hai kích thước đều được chia với một hệ số chung để duy trì tỷ lệ khung hình). Bây giờ chúng tôi hoàn tác việc thay đổi tỷ lệ này để lấy tọa độ của hộp giới hạn trên hình ảnh ban đầu.

```
output[:,1:5] /= scaling_factor
```

Bây giờ, hãy để chúng tôi cắt bất kỳ hộp giới hạn nào có thể có ranh giới bên ngoài hình ảnh vào các cạnh của hình ảnh của chúng tôi.

```
for i in range(output.shape[0]):
    output[i, [1,3]] = torch.clamp(output[i, [1,3]], 0.0, im_dim_list[i,0])
    output[i, [2,4]] = torch.clamp(output[i, [2,4]], 0.0, im_dim_list[i,1])
```

Nếu có quá nhiều ô giới hạn trong hình ảnh, vẽ tất cả chúng bằng một màu có thể không phải là một ý tưởng hay. Tải xuống [tệp này](https://raw.githubusercontent.com/ayooshkathuria/YOLO_v3_tutorial_from_scratch/master/pallete) vào thư mục máy dò của bạn. Đây là tập ngâm gồm nhiều màu để bạn lựa chọn ngẫu nhiên.

```
class_load = time.time()
colors = pkl.load(open("pallete", "rb"))
```

Bây giờ chúng ta hãy viết một hàm để vẽ các hộp.

```
draw = time.time()

def write(x, results, color):
    c1 = tuple(x[1:3].int())
    c2 = tuple(x[3:5].int())
    img = results[int(x[0])]
    cls = int(x[-1])
    label = "{0}".format(classes[cls])
    cv2.rectangle(img, c1, c2,color, 1)
    t_size = cv2.getTextSize(label, cv2.FONT_HERSHEY_PLAIN, 1 , 1)[0]
    c2 = c1[0] + t_size[0] + 3, c1[1] + t_size[1] + 4
    cv2.rectangle(img, c1, c2,color, -1)
    cv2.putText(img, label, (c1[0], c1[1] + t_size[1] + 4), cv2.FONT_HERSHEY_PLAIN, 1, [225,255,255], 1);
    return img
```

Hàm trên vẽ một hình chữ nhật với màu được chọn ngẫu nhiên từ `colors`. Nó cũng tạo ra một hình chữ nhật được tô màu ở góc trên cùng bên trái của hộp giới hạn và ghi lớp của đối tượng được phát hiện trên hình chữ nhật được tô màu. `-1` đối số của hàm `cv2.rectangle` được sử dụng để tạo một hình chữ nhật được tô đầy.

Chúng tôi xác định hàm `write` cục bộ để nó có thể truy cập danh sách `colors`. Chúng tôi cũng có thể bao gồm `colors` như một đối số, nhưng điều đó sẽ cho phép chúng tôi chỉ sử dụng một màu cho mỗi hình ảnh, điều này làm mất đi mục đích của những gì chúng tôi muốn làm.

Khi chúng ta đã xác định chức năng này, bây giờ chúng ta hãy vẽ các hộp giới hạn trên hình ảnh.

```
list(map(lambda x: write(x, loaded_ims), output))
```

Đoạn mã trên sửa đổi các hình ảnh bên trong `load_ims inplace`.

Mỗi hình ảnh được lưu bằng cách thêm tiền tố "`det_`" vào trước tên hình ảnh. Chúng tôi tạo một danh sách các địa chỉ mà chúng tôi sẽ lưu các hình ảnh phát hiện của chúng tôi vào đó.

```
det_names = pd.Series(imlist).apply(lambda x: "{}/det_{}".format(args.det,x.split("/")[-1]))
```

Cuối cùng, ghi các hình ảnh có phát hiện vào địa chỉ trong `det_names`.

```
list(map(cv2.imwrite, det_names, loaded_ims))
end = time.time()
```

## 5.7 Tóm tắt thời gian
Ở cuối trình dò của chúng tôi, chúng tôi sẽ in một bản tóm tắt có chứa phần nào của mã mất bao lâu để thực thi. Điều này rất hữu ích khi chúng ta phải so sánh các siêu thông số khác nhau ảnh hưởng như thế nào đến tốc độ của máy dò. Các siêu tham số như kích thước lô, độ tin cậy của đối tượng và ngưỡng NMS, (được truyền với cờ `bs`, độ tin cậy, `nms_thres`h tương ứng) có thể được đặt trong khi thực hiện script `detection.py` trên dòng lệnh.

```
print("SUMMARY")
print("----------------------------------------------------------")
print("{:25s}: {}".format("Task", "Time Taken (in seconds)"))
print()
print("{:25s}: {:2.3f}".format("Reading addresses", load_batch - read_dir))
print("{:25s}: {:2.3f}".format("Loading batch", start_det_loop - load_batch))
print("{:25s}: {:2.3f}".format("Detection (" + str(len(imlist)) +  " images)", output_recast - start_det_loop))
print("{:25s}: {:2.3f}".format("Output Processing", class_load - output_recast))
print("{:25s}: {:2.3f}".format("Drawing Boxes", end - draw))
print("{:25s}: {:2.3f}".format("Average time_per_img", (end - load_batch)/len(imlist)))
print("----------------------------------------------------------")


torch.cuda.empty_cache()
```

## 5.8 Kiểm tra chương trình phát hiện đối tượng
Ví dụ: chạy trên thiết bị đầu cuối,

```
python detect.py --images dog-cycle-car.png --det det
```

```
Loading network.....
Network successfully loaded
dog-cycle-car.png    predicted in  2.456 seconds
Objects Detected:    bicycle truck dog
----------------------------------------------------------
SUMMARY
----------------------------------------------------------
Task                     : Time Taken (in seconds)

Reading addresses        : 0.002
Loading batch            : 0.120
Detection (1 images)     : 2.457
Output Processing        : 0.002
Drawing Boxes            : 0.076
Average time_per_img     : 2.657
----------------------------------------------------------
```

Một hình ảnh có tên `det_dog-cycle-car.png` được lưu trong thư mục `det`.

![](https://blog.paperspace.com/content/images/2018/04/det_dog-cycle-car.png)

## 5.9 Chạy máy dò trên Video / Webcam
Để chạy trình dò trên video hoặc webcam, mã hầu như vẫn giữ nguyên, ngoại trừ chúng ta không phải lặp lại hàng loạt mà là các khung của video.

Có thể tìm thấy mã để chạy trình dò trên video trong tệp `video.py` trong kho lưu trữ github. Mã này rất giống với mã của `detect.py` ngoại trừ một vài thay đổi.

Đầu tiên, chúng tôi mở nguồn cấp dữ liệu video / camera trong OpenCV.

```
videofile = "video.avi" #or path to the video file. 
cap = cv2.VideoCapture(videofile)  
#cap = cv2.VideoCapture(0)  for webcam
assert cap.isOpened(), 'Cannot capture source'
frames = 0
```

Chúng tôi lặp lại các khung theo cách tương tự như cách chúng tôi lặp qua các hình ảnh.

Rất nhiều mã đã được đơn giản hóa ở nhiều nơi vì chúng ta không còn phải xử lý hàng loạt mà chỉ có một hình ảnh tại một thời điểm. Điều này là do chỉ có một khung hình có thể đến tại một thời điểm. Điều này bao gồm việc sử dụng một tuple thay cho tensor cho `im_dim_list` và một phút thay đổi trong hàm `write`.

Mỗi lần lặp lại, chúng tôi theo dõi số lượng khung hình được chụp trong một biến gọi là `frame`. Sau đó, chúng tôi chia số này cho thời gian trôi qua kể từ khung hình đầu tiên để in FPS của video.

Thay vì ghi các hình ảnh phát hiện vào đĩa bằng `cv2.imwrite`, chúng tôi sử dụng `cv2.imshow` để hiển thị khung với hộp giới hạn được vẽ trên đó. Nếu người dùng nhấn nút `Q`, nó sẽ khiến mã bị đứt vòng lặp và video kết thúc.

```
frames = 0  
start = time.time()

while cap.isOpened():
    ret, frame = cap.read()
    
    if ret:   
        img = prep_image(frame, inp_dim)
#        cv2.imshow("a", frame)
        im_dim = frame.shape[1], frame.shape[0]
        im_dim = torch.FloatTensor(im_dim).repeat(1,2)   
                     
        if CUDA:
            im_dim = im_dim.cuda()
            img = img.cuda()

        output = model(Variable(img, volatile = True), CUDA)
        output = write_results(output, confidence, num_classes, nms_conf = nms_thesh)


        if type(output) == int:
            frames += 1
            print("FPS of the video is {:5.4f}".format( frames / (time.time() - start)))
            cv2.imshow("frame", frame)
            key = cv2.waitKey(1)
            if key & 0xFF == ord('q'):
                break
            continue
        output[:,1:5] = torch.clamp(output[:,1:5], 0.0, float(inp_dim))

        im_dim = im_dim.repeat(output.size(0), 1)/inp_dim
        output[:,1:5] *= im_dim

        classes = load_classes('data/coco.names')
        colors = pkl.load(open("pallete", "rb"))

        list(map(lambda x: write(x, frame), output))
        
        cv2.imshow("frame", frame)
        key = cv2.waitKey(1)
        if key & 0xFF == ord('q'):
            break
        frames += 1
        print(time.time() - start)
        print("FPS of the video is {:5.2f}".format( frames / (time.time() - start)))
    else:
        break     
```

# 6. Kết luận
Trong loạt bài hướng dẫn này, chúng tôi đã triển khai chương trình phát hiện đối tượng từ đầu và cổ vũ cho việc đạt được điều này. Tôi vẫn nghĩ rằng có thể tạo ra mã hiệu quả là một trong những kỹ năng bị đánh giá thấp nhất mà một người học sâu có thể có. Tuy nhiên, bạn có thể nghĩ rằng ý tưởng của bạn mang tính cách mạng, nó không có ích gì trừ khi bạn có thể thử nghiệm nó. Để làm được điều đó, bạn cần phải có kỹ năng viết mã vững vàng.

Tôi cũng đã học được rằng cách tốt nhất để tìm hiểu về bất kỳ chủ đề nào trong học sâu là triển khai mã. Nó buộc bạn phải lướt qua từng phút nhưng tinh tế cơ bản của một chủ đề mà bạn có thể bỏ lỡ khi đọc báo. Tôi hy vọng loạt bài hướng dẫn này sẽ đóng vai trò như một bài tập rèn luyện kỹ năng của bạn với tư cách là một học viên học sâu.